In [ ]:
# One Pickle for NSE

# STATUS: Completed
# Run-time: 2 mins

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=2)

In [ ]:
%%time
import pandas as pd
import numpy as np
import itertools
import datetime
from math import sqrt, exp, log, erf
import os

#... assignments
exchange = 'NSE'
fspath = './zdata/'


putsigma = 1.5
callsigma = 2
maxdte = 65  # max expiry date for options
mindte = 12  # min expiry date for options

tradingdays = 252

blks = 50

#... Get risk-free rate from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
rate = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100

In [ ]:
#... Functions
#_____________

#... Error catching for list comprehension

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan
        
#...function to get historical data
def get_hist(contract, duration):
    '''Gets 1-day bars of contracts for the duration specified
    Args:
        (contract) as obj
        (duration) as int
    Returns: dataframe of symbol, date, ohlc, avg and volume 
    '''
    
    # Prepare the duration
    strduration = str(duration) + ' D'
    
    # Extract the history
    hist = ib.reqHistoricalData(contract=contract, endDateTime='', 
                                    durationStr=strduration, barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)
    
    df = util.df(hist)
    df.insert(0, column='symbol', value=contract.symbol)
    
    return df

#...function to get price and dividend ticker
def get_dividend_ticker(contract):
    '''Gets dividend ticker of the contract
    Arg: (contract) as a qualified contract object with conId
    Returns: ticker'''
    
    ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False) # request ticker stream

    ticker = ib.ticker(contract)
    
    # Ensure the ticker is filled
    while ticker.dividends is None:
        while np.isnan(ticker.marketPrice()):
            ib.sleep(1)

    ib.cancelMktData(contract)
       
    return ticker

#... Black-Scholes
# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undPrice, strike, dte, rate, volatility, divrate):
    ''' Gets Black Scholes output
    Args:
        (undPrice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (dte)      : Days to expiration in float
        (rate)     : dte until expiry in days
        (volatility)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
    sigTsquared = sqrt(dte/365)*volatility
    edivT = exp((-divrate*dte)/365)
    ert = exp((-rate*dte)/365)
    d1 = (log(undPrice*edivT/strike)+(rate+.5*(volatility**2))*dte/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undPrice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undPrice*edivT*iNd1, 2)
    delta = Nd1

    return {'bsmCall': callPrice, 'bsmPut': putPrice, 'bsmDelta': delta}

def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format yyyymmdd
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days

In [22]:
#... build the symbols
#______________________

import pandas as pd
celebrusurl = "http://celebrus.in/displaycapital.php"
df_celeb = pd.read_html(celebrusurl)[0]
df_celeb = df_celeb.rename(columns=df_celeb.iloc[1])[2:]

In [23]:
df_celeb.assign(marginpct = df_celeb['Total Margin'])

,Symbol,Expiry Date,Instrument Type,Lot Size,Initial Margin,Exposure Margin,Total Margin
2,BANKNIFTY,28Feb2019,FUTIDX,20,38912,16596.8,55508.8
3,BANKNIFTY,28Mar2019,FUTIDX,20,39139,16627.8,55766.8
4,BANKNIFTY,31Jan2019,FUTIDX,20,38686,16536.9,55222.9
5,NIFTY,28Feb2019,FUTIDX,75,57377,24438.6,81815.6
6,NIFTY,28Mar2019,FUTIDX,75,57711,24508.7,82219.7


In [ ]:
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct', 'Mlot': 'lot'})

In [ ]:
df_paisa

In [ ]:
# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_paisa
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_paisa.ibSymbol if s not in indexes+discards])

symbols = equities+indexes

In [ ]:
cs = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]

qcs = ib.qualifyContracts(*cs) # qualified underlyings

#****           Single scrip check. To be DELETED in function          *****
#...........................................................................
contract = next(q for q in qcs if q.symbol=='TATACOMM')  # one symbol logic check
#___________________________________________________________________________

In [ ]:
#... get ohlc, with cumulative volatality and standard deviation
#_______________________________________________________________

df_ohlc = get_hist(contract, 365).set_index('date').sort_index(ascending = False)

# get cumulative standard deviation
df_stdev = pd.DataFrame(df_ohlc['close'].expanding(1).std(ddof=0))
df_stdev.columns = ['stdev']

# get cumulative volatility
df_vol = pd.DataFrame(df_ohlc['close'].pct_change().expanding(1).std(ddof=0)*sqrt(tradingdays))
df_vol.columns = ['volatility']

df_ohlc1 = df_ohlc.join(df_vol)

df_ohlc2 = df_ohlc1.join(df_stdev)

#pickle the ohlc
df_ohlc2.to_pickle(fspath+contract.symbol+'_ohlc.pkl')

#... get the underlyings
#_______________________

ticker = get_dividend_ticker(contract)

df_und = util.df([ticker])

cols = ['contract', 'time', 'bid', 'bidSize', 'ask', 'askSize', 'last', 'lastSize', 
        'volume', 'open', 'high', 'low', 'close', 'dividends']
df_und = df_und[cols]

df_und = df_und.assign(undPrice=np.where(df_und['last'].isnull(), df_und.close, df_und['last']))

try: 
    divrate = df_und.dividends[0][0]/df_und.dividends[0][0]/df_und.undPrice
except (TypeError, AttributeError) as e:
    divrate = 0.0

df_und = df_und.assign(divrate=divrate)

df_und = df_und.assign(symbol=[c[1].symbol for c in df_und.contract.items()])

#... get the lot, margin, undPrice and dividend rate
undlot = df_paisa.loc[df_paisa.ibSymbol == contract.symbol, 'lot'].item()
df_und['lot'] = undlot

# margin of underlying
order = Order(action='SELL', totalQuantity=undlot, orderType='MKT')

# margin = float(ib.whatIfOrder(contract, order).initMarginChange)
margin = df_paisa.loc[df_paisa.ibSymbol == contract.symbol, 'TotMgnPerLt'].item()

df_und['margin'] = margin

df_und.to_pickle(fspath+contract.symbol+'_und.pkl')

#... get the options
#___________________

# symbol
symbol = contract.symbol

# rights
right = ['P', 'C'] 

# chains
chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

chain = next(c for c in chains if c.exchange == exchange)

undPrice = df_und.undPrice[0]

strikes = sorted([strike for strike in chain.strikes])
# limit the strikes to outside sigma range
# strikes = sorted([strike for strike in chain.strikes 
#            if (strike < (undPrice - std*putsigma)) | (strike > (undPrice + std*callsigma))])

# limit the expirations to between min and max dates
expirations = sorted([exp for exp in chain.expirations 
                      if mindte < (util.parseIBDatetime(exp)- datetime.datetime.now().date()).days < maxdte])

rights = ['P', 'C']

contracts = [Option(symbol, expiration, strike, right, exchange)
        for right in rights
        for expiration in expirations
        for strike in strikes]

# Eliminate contracts close to the underlying price
dtes = [(util.parseIBDatetime(e) - datetime.datetime.now().date()).days for e in expirations]
sdevs = [df_ohlc2.iloc[i].stdev for i in dtes]


tgts = []
for c in contracts:
    if c.right == 'P':
        if c.strike < (undPrice - df_ohlc2.iloc[get_dte(c.lastTradeDateOrContractMonth)].stdev * putsigma):
            tgts.append(c)
    else:
        if c.strike > (undPrice + df_ohlc2.iloc[get_dte(c.lastTradeDateOrContractMonth)].stdev * callsigma):
            tgts.append(c)

[ib.qualifyContracts(*tgts[i: i+blks]) for i in range(0, len(contracts), blks)]

qc = [c for c in tgts if c.conId != 0]  # qualified option contracts

df_opt = pd.concat([pd.DataFrame(qc, columns=['option']), util.df(qc)], axis=1)

df_opt1 = df_opt[['symbol', 'strike', 'lastTradeDateOrContractMonth', 'right', 'multiplier', 'exchange', 'option']]  # remove unnecessary columns

df_opt1 = df_opt1.rename(columns={'lastTradeDateOrContractMonth': 'expiry'}) # rename expiry column

df_opt1 = df_opt1.assign(undPrice=df_opt1.symbol.map(df_und.set_index('symbol')['undPrice']))

df_opt1['dte'] = (df_opt1.expiry.apply(util.parseIBDatetime) - datetime.datetime.now().date()).dt.days

df_opt1.loc[df_opt1.dte <= 1, 'dte'] = 2 # Make the dte as 2 for 1 day-to-expiry to prevent bsm divide-by-zero error

# get the standard deviation based on days to expiry
df_opt1 = df_opt1.assign(stdev=[df_ohlc2.iloc[i].stdev for i in df_opt1.dte])

# weed out options within threshold of strike and dte
mask = (((df_opt1.right == 'P') & (df_opt1.strike < (df_opt1.undPrice - df_opt1.stdev * putsigma))) | \
       ((df_opt1.right == 'C') & (df_opt1.strike > (df_opt1.undPrice + df_opt1.stdev * callsigma))))

df_opt1 = df_opt1.loc[mask, :].reset_index(drop=True)

# get the volatality based on days to expiry
df_opt1 = df_opt1.assign(volatility=[df_ohlc2.iloc[i].volatility for i in df_opt1.dte])

# get the divrate
divrate = df_und.divrate[0]

bsms = [get_bsm(undPrice, strike, dte, rate, volatility, divrate) 
        for undPrice, strike, dte, rate, volatility, divrate in 
        zip(itertools.repeat(undPrice), df_opt1.strike, df_opt1.dte, itertools.repeat(rate), df_opt1.volatility, itertools.repeat(divrate))]

df_bsm = pd.DataFrame(bsms)

df_opt2 = df_opt1.join(df_bsm)
df_opt2['bsmPrice'] = np.where(df_opt2.right == 'P', df_opt2.bsmPut, df_opt2.bsmCall)
df_opt2['pop'] = np.where(df_opt2.right == 'C', 1-df_opt2.bsmDelta, df_opt2.bsmDelta)
df_opt2 = df_opt2.drop(['bsmCall', 'bsmPut', 'bsmDelta'], axis=1)

# get the option prices
cs = list(df_opt2.option)

# [catch(lambda: ib.reqTickers(c).marketPrice()) for i in range(0, len(cs), 100) for c in cs[i: i+100]]
tickers = [ib.reqTickers(*cs[i: i+100]) for i in range(0, len(cs), 100)]

df_opt3 = df_opt2.assign(price=[t.marketPrice() for ts in tickers for t in ts])

# filter the options whose price is > 0.0
df_opt4 = df_opt3[df_opt3.price > 0.0]

# get the margin of options and make return-on-margin
co = Order(action='SELL', totalQuantity=1, orderType='MKT')
# margins = [float(ib.whatIfOrder(contract, order).initMarginChange) for contract, order in zip(df_opt4.option, itertools.repeat(co))]

df_opt4 = df_opt4.assign(margin=margin)

df_opt4 = df_opt4.assign(rom=df_opt4.price/df_opt4.margin*tradingdays/df_opt4.dte*undlot)

df_opt4 = df_opt4.sort_values(by='rom', ascending=False)

# high and low for the options
df_opt4 = df_opt4.assign(hi52 = df_ohlc2[:252].high.max())
df_opt4 = df_opt4.assign(lo52 = df_ohlc2[:252].low.min())
df_opt4.loc[df_opt4.right == 'P', 'hi52'] = np.nan
df_opt4.loc[df_opt4.right == 'C', 'lo52'] = np.nan

df_opt4.to_pickle(fspath+contract.symbol+'_opt.pkl')

In [ ]:
ib.disconnect()